In [2]:
test_size = 7
datasets = [{
    'name': 'casos_confirmados',
    'path': 'data/covid/casos_confirmados.csv'
}, {
    'name': 'sinteticos',
    'path': 'data/sinteticos/sinteticos.csv'
}, {
    'name': 'temperaturas',
    'path': 'data/temperatura_minima_diaria/temperaturas.csv'
}]
metrics = ['rmse', 'mape', 'mae', 'mpe']

In [3]:
for dataset in datasets:
    for metric in metrics:
        cmd = f"python -m memory_profiler rew_memory.py --dataset-path {dataset['path']} --dataset-name {dataset['name']} --metric {metric}"
        print('-'*120)
        print(f"Dataset: {dataset['name']} | Metric: {metric}")
        !{cmd}

------------------------------------------------------------------------------------------------------------------------
Dataset: casos_confirmados | Metric: rmse
Filename: rew_memory.py

Line #    Mem usage    Increment  Occurences   Line Contents
    17  202.895 MiB  202.895 MiB           1   @profile
    18                                         def profile_memory(ts, thermometer, addr, order, mean, test_size) -> None:
    19  203.480 MiB    0.586 MiB           1       model = RegressionWisardEstimator(ts, thermometer, addr, order=order, mean=mean).fit()
    20  203.480 MiB    0.000 MiB           1       _ = model.forecast(steps=test_size)


------------------------------------------------------------------------------------------------------------------------
Dataset: casos_confirmados | Metric: mape
Filename: rew_memory.py

Line #    Mem usage    Increment  Occurences   Line Contents
    17  202.676 MiB  202.676 MiB           1   @profile
    18                                   